In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Import base classes and helpers
from utils.base import Function
from utils.plot_helpers import plot_loss_curves, plot_decision_boundary

# Import test functions
from utils.test_functions import (
    generate_linear_regression_data, 
    linear_regression_loss,
    generate_logistic_regression_data
)

# Import the optimizers
from optimizers.gradient_descent import MiniBatchGradientDescent
from optimizers.non_differentiable import SubGradientMethod

# Import the models
from models.linear_regression import LinearRegression, RidgeRegression, LassoRegression
from models.logistic_regression import LogisticRegression

# Magic command for plotting
%matplotlib inline

# 1. Linear & Ridge Regression (L2)

First, we'll test `LinearRegression` and `RidgeRegression` on synthetic data. Both use differentiable loss functions, so we can train them with an optimizer like `MiniBatchGradientDescent`.

We expect to see:
- Both models converge.
- The final weights for `RidgeRegression` will be smaller ("shrunken") than for standard `LinearRegression` due to the L2 penalty.

In [ ]:
# --- 1. Generate Data ---
X_lin, Y_lin, W_true_lin = generate_linear_regression_data(N=200, d=5)
print(f"Linear data shape (X): {X_lin.shape}")
print(f"Linear data shape (Y): {Y_lin.shape}")

# --- 2. Instantiate Optimizer ---
# We'll use the same optimizer for both
optim_mb = MiniBatchGradientDescent(alpha=0.01, n_epochs=20, batch_size=32)

# --- 3. Train Linear Regression ---
lin_reg = LinearRegression()
print("\nTraining Standard Linear Regression...")
lin_reg.train(X_lin[:, 1:], Y_lin, optim_mb, is_plot=True) # Pass non-augmented X
print(f"Test Loss (MSE): {lin_reg.test(X_lin[:, 1:], Y_lin):.4f}")

# --- 4. Train Ridge Regression ---
ridge_reg = RidgeRegression(alpha=0.5) # alpha is the regularization strength
print("\nTraining Ridge (L2) Regression...")
ridge_reg.train(X_lin[:, 1:], Y_lin, optim_mb, is_plot=True)
print(f"Test Loss (MSE): {ridge_reg.test(X_lin[:, 1:], Y_lin):.4f}")

# --- 5. Compare Weights ---
print("\n--- Weight Comparison ---")
print(f"True Weights:\n{W_true_lin.flatten()}")
print(f"Linear Reg Weights:\n{lin_reg.W.flatten()}")
print(f"Ridge Reg Weights:\n{ridge_reg.W.flatten()}")

In [ ]:
# Define a loss function for plotting
# We pass the *raw* data, the models handle the bias augmentation
def loss_plotter_lin(W):
    return linear_regression_loss(W, X_lin, Y_lin)

# Plot the loss curves
plot_loss_curves(
    histories={
        "Linear Regression": lin_reg.history,
        "Ridge Regression (L2)": ridge_reg.history
    },
    loss_func_callable=loss_plotter_lin
)

# 2. Lasso Regression (L1)

Next, we'll test `LassoRegression`. This model has a non-differentiable L1 penalty, so we **must** train it with the `SubGradientMethod`.

We expect to see:
- [cite_start]The model converges (though the loss curve may be noisy [cite: 801-810]).
- The final weights have some values that are exactly zero (sparsity).

In [ ]:
# --- 1. Instantiate Optimizer ---
# We must use a sub-gradient optimizer
# [cite_start]A diminishing step size is recommended [cite: 1536-1538]
optim_subgrad = SubGradientMethod(alpha=0.1, n_iterations=1000, policy='diminishing')

# --- 2. Train Lasso Regression ---
lasso_reg = LassoRegression(alpha=0.1) # alpha is the regularization strength
print("\nTraining Lasso (L1) Regression...")
lasso_reg.train(X_lin[:, 1:], Y_lin, optim_subgrad, is_plot=True)
print(f"Test Loss (MSE): {lasso_reg.test(X_lin[:, 1:], Y_lin):.4f}")

# --- 3. Compare Weights ---
print("\n--- Weight Comparison (Lasso) ---")
print(f"True Weights:\n{W_true_lin.flatten()}")
print(f"Lasso Reg Weights:\n{lasso_reg.W.flatten()}")
print("\nNotice how Lasso pushes some weights to zero!")

In [ ]:
# Plot the loss curve for Lasso
# Note: The sub-gradient method is not a descent method,
# so we plot the history of f_best, not the path.
# For simplicity here, we plot the path, but expect it to be noisy.

plot_loss_curves(
    histories={
        "Lasso Regression (L1)": lasso_reg.history
    },
    loss_func_callable=loss_plotter_lin
)

# 3. Logistic Regression (Classification)

Finally, we'll test `LogisticRegression` on a 2D binary classification task. We'll use our new `generate_logistic_regression_data` function.

We expect to see:
- The model trains and achieves high accuracy.
- We can plot a clear decision boundary separating the two classes.

In [ ]:
# --- 1. Generate Data ---
X_log, Y_log = generate_logistic_regression_data(N=300, d=2)

# --- 2. Instantiate Optimizer ---
optim_mb_log = MiniBatchGradientDescent(alpha=0.1, n_epochs=100, batch_size=32)

# --- 3. Train Logistic Regression ---
log_reg = LogisticRegression()
print("\nTraining Logistic Regression...")
log_reg.train(X_log, Y_log, optim_mb_log, is_plot=True) # Pass non-augmented X
print(f"Test Accuracy: {log_reg.test(X_log, Y_log):.4f}")

# --- 4. Plot Decision Boundary ---
# This uses the new helper function
plot_decision_boundary(log_reg, X_log, Y_log, title="Logistic Regression Decision Boundary")

In [ ]:
# Define a loss function for plotting
# This uses the model's internal (private) loss function
def loss_plotter_log(W):
    X_aug = log_reg._add_bias(X_log)
    return log_reg._loss(W.reshape(-1, 1), X_aug, Y_log)

# Plot the loss curves
plot_loss_curves(
    histories={
        "Logistic Regression": log_reg.history
    },
    loss_func_callable=loss_plotter_log
)